In [5]:
import tensorflow as tf
import numpy as np

## 1. Lists

In [20]:
l = []

@tf.function
def f(x):
    for i in x:
        l.append(i + 1)

In [22]:
f(tf.constant([1, 2, 3]))
print(l)


[<tf.Tensor 'while/add:0' shape=() dtype=int32>]


Let's rewrite this function with the use of `TensorArray` which is a data structure for dynamic arrays.
use its `write` and `stack` method to rewrite the above function.\
https://www.tensorflow.org/api_docs/python/tf/TensorArray#write

https://www.tensorflow.org/api_docs/python/tf/TensorArray#stack

In [104]:
@tf.function
def f(x):
    ta = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    for i in tf.range(tf.shape(x)[0]):
        # Write your code here
        tf.print(i)
        ta = ta.write(i,x[i])
    return ta.stack()

In [108]:
f(tf.constant([1, 2, 3]))

0
1
2


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3])>

## 2. Tensor Objects and Numpy Arrays

In [112]:
def np_multiply(a, b):
    return np.multiply(a, b)

In [114]:
a_tensor = tf.constant([5])
b_tensor = tf.constant([10])
np_multiply(a_tensor, b_tensor)

array([50])

In [130]:
import tensorflow.experimental.numpy as tnp

@tf.function
def tf_multiply(a, b):
    #write your code here
    #return tnp.multiply(a, b)
    return tf.math.multiply(a, b)

In [132]:
a_array = np.array([5])
b_array = np.array([10])
a_tensor = tf.constant([5])
b_tensor = tf.constant([10])
tf_multiply(a_array, b_array)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([50])>

In [190]:
# تعریف تابع گرافی برای np_multiply
@tf.function
def np_multiply_tf_func(a, b):
    return tnp.multiply(a, b)

# تعریف تابع گرافی برای tf_multiply
@tf.function
def tf_multiply_tf_func(a, b):
    return tf.math.multiply(a, b)

now convert there two functions into Graph functions and run again:

In [184]:
np_multiply_tf_func = np.multiply(a_array, b_array)
tf_multiply_tf_func = tnp.multiply(a_array, b_array)

In [192]:
np_multiply_tf_func(a_tensor, b_tensor)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([50])>

As you noticed we can't use `numpy` operations inside graph functions and we must use their `TensorFlow` equivalents.\
Also TensorFlow has a new API which implemented the numpy functions:
https://www.tensorflow.org/guide/tf_numpy

In [161]:
tf_multiply_tf_func(a_array, b_array)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([50])>

## 3. Tensor Assignment

In [194]:
def assign_one(x):
    x[1] = 1
    return x

In [196]:
x = tf.constant([5, 6, 7])
assign_one(x)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

You can see that TensorFlow `Tensor` object does not support item assignment in contrast with numpy array or python lists

But there is a way to do this in eager mode:
1. create a numpy array from the tensor by calling `tensor.numpy()
2. assign the value to numpy array 
3. create a constant tensor from the numpy array

let's write it

In [199]:
def assign_one_new(x):
    x_numpy = x.numpy()
    #Write your code here
    x_numpy[0] = 1
    return tf.constant(x_numpy)

In [201]:
x = tf.constant([5, 6, 7])
assign_one_new(x)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 6, 7])>

Now convert this function to a Graph function

In [281]:
# ایجاد tf.Variable خارج از تابع
v = tf.Variable([0, 0, 0], dtype=tf.int32)  # مقدار اولیه می‌تواند هر چیزی باشد

def assign_one_new(x):
    # به‌روزرسانی متغیر با مقدار ورودی x
    v.assign(x)
    # تخصیص مقدار 1 به ایندکس 1
    v.assign(tf.tensor_scatter_nd_update(v, [[1]], [1]))
    return v

In [283]:
assign_one_graph = tf.function(assign_one_new)

In [285]:
# تنسور ورودی
x = tf.constant([5, 6, 7])

result_graph = assign_one_graph(x)

# فراخوانی تابع
result = assign_one_new(x)
print(result_graph) 
print(result) 

tf.Tensor([5 1 7], shape=(3,), dtype=int32)
<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([5, 1, 7])>


You can see that there is no way to assign a value to a `Tensor` in graph mode, you should consider it in your implementations.
You can use `tf.Variable` instead of Tensors for assignments:
https://www.tensorflow.org/api_docs/python/tf/Variable#assign